In [48]:
import os
import json
import matplotlib.pyplot as plt
import pandas as pd
import glob
import re
from datetime import datetime

In [55]:
# todo: transform into a sqlalchemy method which dumps the data into a table

def get_json_files(strategy_name):
    # Define the directory path for the strategy
    strategy_dir = os.path.join('../strategy', strategy_name, 'backtests')
    # todo: replace for strategy in production
    # strategy_dir = os.path.join('/strategy', strategy_name, 'backtests')
    
    # Get all directories under the strategy directory
    dirs = [d for d in os.listdir(strategy_dir) if os.path.isdir(os.path.join(strategy_dir, d))]
    # Sort the directories by date
    dirs.sort(key=lambda x: datetime.strptime(x, '%Y-%m-%d_%H-%M-%S'), reverse=True)
    # Get the newest directory
    newest_dir = dirs[0]
    # Use glob to find the JSON file that matches the pattern in the newest directory
    json_files = glob.glob(os.path.join(strategy_dir, newest_dir, '*.json'))
    # Define the pattern for the JSON file
    pattern = re.compile(r'\d{10}\.json$')
    # Filter the json_files list to only include files that match the pattern
    json_files = [f for f in json_files if pattern.match(os.path.basename(f))]
    # Since we know there's only one file that matches the pattern, we can just return the first item in the list
    return json_files[0]

In [56]:
file_dir = get_json_files('ib_loader')

In [72]:
# todo: transform into a sqlalchemy method which dumps the data into a table

# testing
def print_first_nodes(json_file):
    # Open and load the JSON file
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Print the first level keys (nodes) of the JSON object
    for key in data.keys():
        if key == 'Statistics' or key == 'RunTimeStatistics':
            for sub_key in data[key].items():
                print(f'{sub_key[0]}: {sub_key[1]}')
        
            

# Use the function
print_first_nodes(file_dir)

Total Trades: 1
Average Win: 0%
Average Loss: 0%
Compounding Annual Return: 2.246%
Drawdown: 15.800%
Expectancy: 0
Net Profit: 2.053%
Sharpe Ratio: -0.099
Sortino Ratio: -0.12
Probabilistic Sharpe Ratio: 17.252%
Loss Rate: 0%
Win Rate: 0%
Profit-Loss Ratio: 0
Alpha: 0.033
Beta: 1.235
Annual Standard Deviation: 0.186
Annual Variance: 0.035
Information Ratio: 0.153
Tracking Error: 0.15
Treynor Ratio: -0.015
Total Fees: $797.34
Estimated Strategy Capacity: $0
Lowest Capacity Asset: BHP.IBData 2S
Portfolio Turnover: 0.29%


In [66]:
def parse_json(json_file):
    # Open and load the JSON file
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Extract the "RollingWindow" data
    rolling_window_data = data['RollingWindow']

    # Initialize a list to store the data for each period
    periods_data = []

    # For each period in the "RollingWindow" data
    for period, period_data in rolling_window_data.items():
        # Extract the "TradeStatistics" and "PortfolioStatistics" data
        trade_stats = period_data['TradeStatistics']
        portfolio_stats = period_data['PortfolioStatistics']

        # Combine the "TradeStatistics" and "PortfolioStatistics" data
        combined_data = {**trade_stats, **portfolio_stats}

        # Add the period as a new key-value pair
        combined_data['Period'] = period

        # Add the combined data to the list of periods data
        periods_data.append(combined_data)

    # Create a DataFrame from the periods data
    df = pd.DataFrame(periods_data)

    return df